In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the training data
train_input_texts = pickle.load(open('DS_5_train_input', 'rb'))
train_output_texts = pickle.load(open('DS_5_train_output', 'rb'))

In [4]:
text_pairs = []
for line_ip, line_op in zip(train_input_texts, train_output_texts):
    
    output_lng = "[start] " + line_op + " [end]"
    text_pairs.append((line_ip, output_lng))

In [9]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.20 * len(text_pairs))
# num_train_samples = len(text_pairs) - 2*num_val_samples
num_train_samples = len(text_pairs) - num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
#test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [11]:
len(train_pairs)

4000

In [12]:
#Vectorizing the Language_1 and Language_2 text pairs

import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 40
sequence_length = 200

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_lang1_texts = [pair[0] for pair in train_pairs]
train_lang2_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_lang1_texts)
target_vectorization.adapt(train_lang2_texts)

In [13]:
#Preparing datasets for the translation task

batch_size = 64

def format_dataset(lang1, lang2):
    lang1 = source_vectorization(lang1)
    lang2 = target_vectorization(lang2)
    return ({
        "language_1": lang1,
        "language_2": lang2[:, :-1],
    }, lang2[:, 1:])

def make_dataset(pairs):
    lang1_texts, lang2_texts = zip(*pairs)
    lang1_texts = list(lang1_texts)
    lang2_texts = list(lang2_texts)
    dataset = tf.data.Dataset.from_tensor_slices((lang1_texts, lang2_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [14]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['language_1'].shape: {inputs['language_1'].shape}")
    print(f"inputs['language_2'].shape: {inputs['language_2'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['language_1'].shape: (64, 200)
inputs['language_2'].shape: (64, 200)
targets.shape: (64, 200)


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [17]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [18]:
#PositionalEmbedding layer

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [19]:

#End-to-end Transformer

embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="language_1")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="language_2")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [20]:
#Training the sequence-to-sequence Transformer
from tensorflow.keras.optimizers import Adam

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="transformer_model.keras",
        save_best_only=True,
        monitor="loss")]

transformer.compile(
    optimizer=Adam(),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=callbacks)

Epoch 1/100
63/63 [==============================] - 39s 378ms/step - loss: 2.5840 - accuracy: 0.2510 - val_loss: 1.6405 - val_accuracy: 0.3727
Epoch 2/100
63/63 [==============================] - 22s 345ms/step - loss: 1.6795 - accuracy: 0.3721 - val_loss: 1.4015 - val_accuracy: 0.4537
Epoch 3/100
63/63 [==============================] - 22s 349ms/step - loss: 1.4299 - accuracy: 0.4452 - val_loss: 1.2427 - val_accuracy: 0.5095
Epoch 4/100
63/63 [==============================] - 23s 371ms/step - loss: 1.2788 - accuracy: 0.5013 - val_loss: 1.1976 - val_accuracy: 0.5279
Epoch 5/100
63/63 [==============================] - 24s 377ms/step - loss: 1.1777 - accuracy: 0.5368 - val_loss: 1.1108 - val_accuracy: 0.5559
Epoch 6/100
63/63 [==============================] - 23s 359ms/step - loss: 1.0582 - accuracy: 0.5759 - val_loss: 0.9855 - val_accuracy: 0.5997
Epoch 7/100
63/63 [==============================] - 22s 353ms/step - loss: 1.1272 - accuracy: 0.5636 - val_loss: 0.9067 - val_accuracy:

In [ ]:



# #Translating new sentences with our Transformer model

# import numpy as np
# spa_vocab = target_vectorization.get_vocabulary()
# spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
# max_decoded_sentence_length = 200

# def decode_sequence(input_sentence):
#     tokenized_input_sentence = source_vectorization([input_sentence])
#     decoded_sentence = "[start]"
#     for i in range(max_decoded_sentence_length):
#         tokenized_target_sentence = target_vectorization(
#             [decoded_sentence])[:, :-1]
#         predictions = transformer(
#             [tokenized_input_sentence, tokenized_target_sentence])
#         sampled_token_index = np.argmax(predictions[0, i, :])
#         sampled_token = spa_index_lookup[sampled_token_index]
#         decoded_sentence += " " + sampled_token
#         if sampled_token == "[end]":
#             break
#     return decoded_sentence

In [ ]:
# decoded_texts = []  
# test_eng_texts = [pair[0] for pair in test_pairs]

# for input_sentence in test_eng_texts:
#   decoded_sentence = decode_sequence(input_sentence)
      
#   decoded_sentence = decoded_sentence.replace('[start] ', '').replace(' [end]', '')
#   decoded_texts.append(decoded_sentence)

In [ ]:
# total_words = 0
# matching = 0
# language2_expected_output = [pair[1] for pair in test_pairs]


# for i, j in zip(language2_expected_output, decoded_texts):
#   i = i.replace('[start] ', '').replace(' [end]', '')
#   list1 = i.split()
#   list2 = j.split()

#   total_words += len(list1)

#   for word1, word2 in zip(list1, list2):
#     if(word1 == word2):
#       matching += 1

In [ ]:
# print(matching/total_words)


**Running the model and decoding text for test dataset**

Note: In order to load the model and run the test data on it, please run all the cells of this Jupyter notebook, except the cell which runs model.fit method to train the model. This is because there are many preprocessing steps involved before decoding can be done on test data.

In [21]:
def testing(model, test_set):

  spa_vocab = target_vectorization.get_vocabulary()
  spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
  max_decoded_sentence_length = 200

  def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = model(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

  
  decoded_texts = []  
  for input_sentence in test_set:
    decoded_sentence = decode_sequence(input_sentence)
      
    decoded_sentence = decoded_sentence.replace('[start] ', '').replace(' [end]', '')
    decoded_texts.append(decoded_sentence)

  return decoded_texts

In [23]:
#Please run the below code for testing
import pickle
import keras

#Test file
test_input_texts = pickle.load(open('DS_5_test_input', 'rb'))

#the keras model sent in email for the project
model = keras.models.load_model(
    "transformer_model.keras",
    custom_objects={"TransformerDecoder": TransformerDecoder,
                    "TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})

#decoded_texts = testing(model, test_input_texts)



Decoding test data in sets of 1000 because of GPU crashes

In [24]:
decoded_texts_1 = testing(model, test_input_texts[:1000])

In [25]:
pickle.dump(decoded_texts_1, open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts1000','wb')) 

In [ ]:
len(decoded_texts_1)

In [26]:
decoded_texts_2 = testing(model, test_input_texts[1000:2000])

In [27]:
pickle.dump(decoded_texts_2, open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts2000','wb')) 

In [ ]:
decoded_texts_1

In [30]:
decoded_texts_3 = testing(model, test_input_texts[2000:3000])

In [31]:
pickle.dump(decoded_texts_3, open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts3000','wb')) 

In [32]:
decoded_texts_4 = testing(model, test_input_texts[3000:4000])

In [33]:
pickle.dump(decoded_texts_4, open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts4000','wb')) 

In [34]:
decoded_texts_5 = testing(model, test_input_texts[4000:5000])

In [35]:
pickle.dump(decoded_texts_5, open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts5000','wb')) 

Loading all decoded texts into one file

In [37]:
decoded_text_2000 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts2000','rb'))

In [36]:
decoded_text_1000 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts1000','rb'))

In [38]:
decoded_text_3000 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts3000','rb')) 

In [39]:
decoded_text_4000 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts4000','rb')) 

In [40]:
decoded_text_5000 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/project2/translated_test_texts5000','rb')) 

In [41]:
decoded_list = decoded_text_1000 + decoded_text_2000 + decoded_text_3000 + decoded_text_4000 + decoded_text_5000

In [42]:
len(decoded_list)

5000

In [ ]:
decoded_list

In [43]:
pickle.dump(decoded_list, open('/content/drive/MyDrive/Colab Notebooks/project2/final_output_data','wb')) 